In [ ]:
from llama_index.tools.waii import WaiiToolSpec

waii_tool = WaiiToolSpec(
    url="https://tweakit.waii.ai/api/",
    # API Key of Waii (not OpenAI API key)
    api_key="3........",
    # Which database you want to use, you need add the db connection to Waii first
    database_key="snowflake://....",
    verbose=True,
)

In [ ]:
from llama_index import VectorStoreIndex

# Use as Data Loader, load data to index and query it
documents = waii_tool.load_data("Get all tables with their number of columns")
index = VectorStoreIndex.from_documents(documents).as_query_engine()

index.query(
    "Which table contains most columns, tell me top 5 tables with number of columns?"
).response

.

'SELECT\n    table_schema,\n    table_name,\n    COUNT(column_name) AS number_of_columns\nFROM waii.information_schema.columns\nGROUP BY\n    table_schema,\n    table_name\nORDER BY\n    table_schema,\n    table_name\n'

..

,TABLE_SCHEMA,TABLE_NAME,NUMBER_OF_COLUMNS
0,BATTLE_DEATH,BATTLE,6
1,BATTLE_DEATH,DEATH,5
2,BATTLE_DEATH,SHIP,7
3,CAR,CARS_DATA,8
4,CAR,CAR_MAKERS,4
...,...,...,...
107,VOTER,VOTES,5
108,WORLD,CITY,5
109,WORLD,COUNTRY,15
110,WORLD,COUNTRYLANGUAGE,4


"The table 'COLUMNS' contains the most columns. The top 5 tables with the number of columns are 'COLUMNS' with 43 columns, 'TABLES' with 25 columns, and the remaining tables have fewer than 25 columns."

In [ ]:
# Use as tool, initialize it
from llama_index.agent import OpenAIAgent
from llama_index.llms import OpenAI

agent = OpenAIAgent.from_tools(
    waii_tool.to_tool_list(), llm=OpenAI(model="gpt-4-1106-preview"), verbose=True
)

In [ ]:
# Ask simple questions
print(agent.chat("Give me top 3 countries with the most number of car factory"))

STARTING TURN 1
---------------

=== Calling Function ===
Calling function: get_answer with args: {"ask":"What are the top 3 countries with the highest number of car factories?"}
.

'WITH carmakerscountries AS (\n    SELECT\n        countryname,\n        COUNT(id) AS numberoffactories\n    FROM waii.car.car_makers\n    INNER JOIN waii.car.countries\n        ON country = countryid\n    GROUP BY\n        countryname\n)\n\nSELECT\n    countryname,\n    numberoffactories\nFROM carmakerscountries\nORDER BY\n    numberoffactories DESC\nLIMIT 3\n'

.

,COUNTRYNAME,NUMBEROFFACTORIES
0,japan,5
1,germany,4
2,usa,4


Got output: Japan has 5 factories, Germany has 4 factories, and the USA also has 4 factories.

STARTING TURN 2
---------------

The top 3 countries with the highest number of car factories are:

1. Japan with 5 factories
2. Germany with 4 factories
3. USA with 4 factories


In [ ]:
print(agent.chat("What are the car factories of these countries"))

STARTING TURN 1
---------------

=== Calling Function ===
Calling function: get_answer with args: {"ask": "What are the car factories in Japan?"}
.

"SELECT DISTINCT fullname\nFROM waii.car.car_makers\nINNER JOIN waii.car.countries\n    ON country = countryid\nWHERE\n    countryname ILIKE '%japan%'\n"

.

,FULLNAME
0,Nissan Motors
1,Honda
2,Mazda
3,Subaru
4,Toyota


Got output: The result of the information from multiple sources is a list of car manufacturers. The list includes Nissan Motors, Honda, Mazda, Subaru, and Toyota.

=== Calling Function ===
Calling function: get_answer with args: {"ask": "What are the car factories in Germany?"}
.

...

Got output: The result of the information from multiple sources is that there are four different car makers mentioned: Volkswagen, BMW, Daimler Benz, and Opel.

=== Calling Function ===
Calling function: get_answer with args: {"ask": "What are the car factories in the USA?"}
.

"SELECT DISTINCT maker\nFROM waii.car.car_makers\nINNER JOIN waii.car.countries\n    ON country = countryid\nWHERE\n    countryname ILIKE '%usa%'\n"

.

,MAKER
0,amc
1,gm
2,ford
3,chrysler


Got output: The result of the information from multiple sources is that there are four different car makers mentioned: AMC, GM, Ford, and Chrysler.

STARTING TURN 2
---------------

Here are the car factories in the top 3 countries with the most factories:

**Japan:**
- Nissan Motors
- Honda
- Mazda
- Subaru
- Toyota

**Germany:**
- Volkswagen
- BMW
- Daimler Benz
- Opel

**USA:**
- AMC
- GM (General Motors)
- Ford
- Chrysler


In [ ]:
# Do performance analysis
print(
    agent.chat(
        "Give me top 3 longest running queries, include the complete query_id and their duration. And analyze performance of the first query"
    )
)

STARTING TURN 1
---------------

=== Calling Function ===
Calling function: get_answer with args: {"ask":"What are the top 3 longest running queries including their query_id and duration?"}
.

'SELECT\n    query_id,\n    total_elapsed_time\nFROM TABLE(INFORMATION_SCHEMA.QUERY_HISTORY(RESULT_LIMIT => 10000))\nORDER BY\n    total_elapsed_time DESC NULLS LAST\nLIMIT 3\n'

.......

,QUERY_ID,TOTAL_ELAPSED_TIME
0,01b0c491-0001-ec2a-0022-ba8700c79812,27117
1,01b0c5df-0001-ebd9-0022-ba8700c7d18a,15903
2,01b0cb83-0001-edf6-0022-ba8700c87192,11813


Got output: The result from multiple sources shows the total elapsed time for each query. The first query, with the ID '01b0c491-0001-ec2a-0022-ba8700c79812', had a total elapsed time of 27117. The second query, with the ID '01b0c5df-0001-ebd9-0022-ba8700c7d18a', had a total elapsed time of 15903. Finally, the third query, with the ID '01b0cb83-0001-edf6-0022-ba8700c87192', had a total elapsed time of 11813.

STARTING TURN 2
---------------

=== Calling Function ===
Calling function: performance_analyze with args: {"query_uuid": "01b0c491-0001-ec2a-0022-ba8700c79812"}
Got output: {
  "summary": [
    "The 'tablescan' operator on 'tweakit_playground.retail_data.store_sales' is the most time-consuming, with an overall percentage of execution time of 0.78, processing time of 0.01, and remote disk IO of 0.76. It scanned 2,731,773,952 bytes and emitted 110,005,394 output rows.",
    "The 'aggregate' operator associated with the 'GROUP BY ss_customer_sk, d_year, d_moy' clause and SUM functio

In [ ]:
# Diff two queries
previous_query = """
SELECT
    employee_id,
    department,
    salary,
    AVG(salary) OVER (PARTITION BY department) AS department_avg_salary,
    salary - AVG(salary) OVER (PARTITION BY department) AS diff_from_avg
FROM
    employees;
"""
current_query = """
SELECT
    employee_id,
    department,
    salary,
    MAX(salary) OVER (PARTITION BY department) AS department_max_salary,
    salary - AVG(salary) OVER (PARTITION BY department) AS diff_from_avg
FROM
    employees;
LIMIT 100;
"""
print(agent.chat(f"tell me difference between {previous_query} and {current_query}"))

STARTING TURN 1
---------------

=== Calling Function ===
Calling function: diff_query with args: {
  "previous_query": "SELECT\n    employee_id,\n    department,\n    salary,\n    AVG(salary) OVER (PARTITION BY department) AS department_avg_salary,\n    salary - AVG(salary) OVER (PARTITION BY department) AS diff_from_avg\nFROM\n    employees;",
  "current_query": "SELECT\n    employee_id,\n    department,\n    salary,\n    MAX(salary) OVER (PARTITION BY department) AS department_max_salary,\n    salary - AVG(salary) OVER (PARTITION BY department) AS diff_from_avg\nFROM\n    employees;\nLIMIT 100;"
}
Got output: The query retrieves the maximum salary and the difference from the average salary for each employee's department. It does this by retrieving the employee ID, department, and salary from the employees table. Then, it calculates the maximum salary for each department using the MAX() function and the PARTITION BY clause. Next, it calculates the difference between each employee's s

In [ ]:
# Describe dataset
print(agent.chat("Summarize the dataset"))

STARTING TURN 1
---------------

=== Calling Function ===
Calling function: describe_dataset with args: {"ask":"Summarize the dataset"}
Got output: The dataset consists of multiple schemas, each containing tables related to different domains. The "FLIGHT" schema contains tables related to airlines, airports, and flights. It provides information about different airlines, airports, and flight details. The "STUDENT_TRANSCRIPTS_TRACKING" schema contains tables related to student information, courses, degree programs, departments, semesters, and transcripts. It provides a comprehensive database for tracking and managing student records, enrollment, and academic information. The "WORLD" schema contains tables related to cities, countries, and languages. It provides information about cities, countries, their populations, languages spoken, and various other attributes. The "INFORMATION_SCHEMA" schema provides information about the database objects and metadata in the WAII database. It includes

In [ ]:
q = """
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, lag, lead, round
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("yearly_car_analysis").getOrCreate()

yearly_avg_hp = cars_data.groupBy("year").agg(avg("horsepower").alias("avg_horsepower"))

windowSpec = Window.orderBy("year")

yearly_comparisons = yearly_avg_hp.select(
    "year",
    "avg_horsepower",
    lag("avg_horsepower").over(windowSpec).alias("prev_year_hp"),
    lead("avg_horsepower").over(windowSpec).alias("next_year_hp")
)

final_result = yearly_comparisons.select(
    "year",
    "avg_horsepower",
    round(
        (yearly_comparisons.avg_horsepower - yearly_comparisons.prev_year_hp) / 
        yearly_comparisons.prev_year_hp * 100, 2
    ).alias("percentage_diff_prev_year"),
    round(
        (yearly_comparisons.next_year_hp - yearly_comparisons.avg_horsepower) / 
        yearly_comparisons.avg_horsepower * 100, 2
    ).alias("percentage_diff_next_year")
).orderBy("year")

final_result.show()
"""
print(agent.chat(f"translate this pyspark query {q}, to Snowflake"))

STARTING TURN 1
---------------

=== Calling Function ===
Calling function: transcode with args: {"instruction":"Translate this PySpark query to Snowflake SQL.","source_dialect":"spark","source_query":"from pyspark.sql import SparkSession\nfrom pyspark.sql.functions import avg, lag, lead, round\nfrom pyspark.sql.window import Window\n\nspark = SparkSession.builder.appName(\"yearly_car_analysis\").getOrCreate()\n\nyearly_avg_hp = cars_data.groupBy(\"year\").agg(avg(\"horsepower\").alias(\"avg_horsepower\"))\n\nwindowSpec = Window.orderBy(\"year\")\n\nyearly_comparisons = yearly_avg_hp.select(\n    \"year\",\n    \"avg_horsepower\",\n    lag(\"avg_horsepower\").over(windowSpec).alias(\"prev_year_hp\"),\n    lead(\"avg_horsepower\").over(windowSpec).alias(\"next_year_hp\")\n)\n\nfinal_result = yearly_comparisons.select(\n    \"year\",\n    \"avg_horsepower\",\n    round(\n        (yearly_comparisons.avg_horsepower - yearly_comparisons.prev_year_hp) / \n        yearly_comparisons.prev_year

In [ ]:
print(agent.chat("Run it"))

STARTING TURN 1
---------------

=== Calling Function ===
Calling function: run_query with args: {"sql":"WITH yearly_avg_hp AS (\n    SELECT\n        year,\n        AVG(horsepower) AS avg_horsepower\n    FROM waii.car.cars_data\n    GROUP BY\n        year\n),\n\nyearly_comparisons AS (\n    SELECT\n        year,\n        avg_horsepower,\n        LAG(avg_horsepower) OVER (ORDER BY year) AS prev_year_hp,\n        LEAD(avg_horsepower) OVER (ORDER BY year) AS next_year_hp\n    FROM yearly_avg_hp\n)\n\nSELECT\n    year,\n    avg_horsepower,\n    ROUND((\n        (\n            avg_horsepower - prev_year_hp\n        ) / NULLIF(prev_year_hp, 0) * 100\n    ), 2) AS percentage_diff_prev_year,\n    ROUND((\n        (\n            next_year_hp - avg_horsepower\n        ) / NULLIF(avg_horsepower, 0) * 100\n    ), 2) AS percentage_diff_next_year\nFROM yearly_comparisons\nORDER BY\n    year"}
..

,YEAR,AVG_HORSEPOWER,PERCENTAGE_DIFF_PREV_YEAR,PERCENTAGE_DIFF_NEXT_YEAR
0,1970,148.857143,NaN,-29.51
1,1971,104.928571,-29.51,14.53
2,1972,120.178571,14.53,8.57
3,1973,130.475000,8.57,-27.78
4,1974,94.230769,-27.78,7.25
5,1975,101.066667,7.25,0.05
6,1976,101.117647,0.05,3.91
7,1977,105.071429,3.91,-5.12
8,1978,99.694444,-5.12,1.52
9,1979,101.206897,1.52,-23.44


Got output: [Document(id_='2a53eaf3-ffa3-4ce0-9d7d-60575ae77f63', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='721222e7ca4249d174a35e130eb622c0c9a7b46211a27f7998008ee9aeff31a6', text="{'YEAR': 1970, 'AVG_HORSEPOWER': 148.857143, 'PERCENTAGE_DIFF_PREV_YEAR': None, 'PERCENTAGE_DIFF_NEXT_YEAR': -29.51}", start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), Document(id_='750de6d2-6872-4d48-bbce-65c6ae020c14', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='f8035cca9f3954c9b453303c9677871c23741ecff32d5070f30e579fa8d38e10', text="{'YEAR': 1971, 'AVG_HORSEPOWER': 104.928571, 'PERCENTAGE_DIFF_PREV_YEAR': -29.51, 'PERCENTAGE_DIFF_NEXT_YEAR': 14.53}", start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {val